In [ ]:
import os
import numpy as np
import pandas as pd
import json
import tensorflow as tf
from datetime import datetime, timedelta

from dotenv import load_dotenv

from alpaca.data.requests import StockBarsRequest
from alpaca.data.historical.stock import StockHistoricalDataClient
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Reshape

import matplotlib.pyplot as plt

import hvplot.pandas

In [ ]:
# Load .env file

load_dotenv()

In [ ]:
# Set Alpaca API key and secret

ALPACA_API_KEY = os.getenv("ALPACA_API_KEY")
ALPACA_SECRET_KEY = os.getenv("ALPACA_API_SECRET")
client = StockHistoricalDataClient(ALPACA_API_KEY, ALPACA_SECRET_KEY)

# Create the Alpaca API object

timeframe = TimeFrame(1, TimeFrameUnit.Day)
symbol = 'SPY'
start = datetime.utcnow() - timedelta(days=3650)
end=datetime.utcnow() - timedelta(days=730)
request = StockBarsRequest(symbol_or_symbols=symbol, start=start, end=end, timeframe=timeframe)

In [ ]:
source_df = client.get_stock_bars(request).df.tz_convert('America/New_York', level=1)

In [ ]:
display(source_df.head())
display(source_df.tail())

In [ ]:
bars_df = source_df.copy()
# pct_change is profit from last close
bars_df["pct_change"] = bars_df["close"].pct_change()
# signal for when we want to be in or out of a stock
#bars_df["signal"] = np.where(bars_df["pct_change"] > 0, 1.0, 0.0)
# reaction is the signal diff
#bars_df["reaction"] = bars_df["signal"].diff()
# action is if we could perfectly predict the next close
#bars_df["action"] = bars_df["reaction"].shift(-1)
# these values are the high, low, and open as a percentage of the current close
bars_df["high %"] = (bars_df["high"] - bars_df["close"])/bars_df["close"]
bars_df["low %"] = (bars_df["low"] - bars_df["close"])/bars_df["close"]
bars_df["open %"] = (bars_df["open"] - bars_df["close"])/bars_df["close"]

In [ ]:
bars_df.info()

In [ ]:
# Cleanup DF for model

bars_df = bars_df.droplevel(level=0).dropna()
bars_df.head()

In [ ]:
# Normalize data set

scaler = StandardScaler()
bars_df_scaled = scaler.fit_transform(bars_df)


In [ ]:
# Define constants

# input shape

input_shape = bars_df_scaled.shape[1]
latent_dim = 11
num_samples = bars_df_scaled.shape[0]
batch_size = 16


In [ ]:
# Define generator (Sequential)

build_generator = Sequential([
    Dense(128,input_shape=(input_shape,), activation="relu"),
    Dense(256, activation="relu"),
    Dense(512, activation="relu"),
    Dense(input_shape, activation="sigmoid")
])

# Define descriminator

build_discriminator = Sequential([
    Dense(512, input_shape=(input_shape,), activation="relu"),
    Dense(256, activation="relu"),
    Dense(128, activation="relu"),
    Dense(1, activation="sigmoid")
])

In [ ]:
# Compile generator
build_generator.compile(loss = "mse", optimizer="adam")

# Compile discriminator
build_discriminator.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Combine models

# Confirm that discriminator weights are not training during generators training

build_discriminator.trainable=False

# Setup for generators data

z = tf.keras.Input(shape=(latent_dim,))

# Generated data by the generator

generated_data = build_generator(z)

# Discriminators verdict

validity = build_discriminator(generated_data)

In [ ]:
# Define combined models

combined = tf.keras.Model(z, validity)
combined.compile(optimizer="adam", loss="mse", metrics=["accuracy"])

# Print summary of the combined model
combined.summary()

In [ ]:
# Define training loop

epochs = 10000

generator_weights_path = "./generator_model.h5"
discriminator_weights_path = "./discriminator_model.h5"

for epoch in range(epochs):

    # Train discriminator
    # Sample real data
    idx = np.random.choice(num_samples, batch_size, replace=False)
    real_data = bars_df_scaled[idx]

 # Generate fake data (Random Walk noise)

    gaussian_noise = np.random.normal(0,1,size=(batch_size,input_shape))
    random_walk_noise = np.cumsum(gaussian_noise, axis=0)

    fake_data = build_generator.predict(random_walk_noise)

    # Train discriminator

    d_loss_real = build_discriminator.train_on_batch(real_data, np.ones((batch_size, 1)))
    d_loss_fake = build_discriminator.train_on_batch(fake_data, np.zeros((batch_size, 1)))
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    
    # Train generator 

    noise = np.random.normal(0, 1, (batch_size, input_shape))
    g_loss = combined.train_on_batch(noise, np.ones((batch_size, 1)))
    
    # Print progress

    print(f"Epoch {epoch}, Discriminator Loss: {d_loss}, Generator Loss: {g_loss}")


In [ ]:
# Saving Model Weights

build_generator.save("generator_model.h5")
build_discriminator.save("discriminator_model.h5")

In [ ]:
# Create alpaca object for testing

timeframe = TimeFrame(1, TimeFrameUnit.Day)
symbol = 'SPY'
start = datetime.utcnow() - timedelta(days=730)
end=datetime.utcnow() - timedelta(days=1)
request = StockBarsRequest(symbol_or_symbols=symbol, start=start, end=end, timeframe=timeframe)

In [ ]:
# Create df for testing

test_df = client.get_stock_bars(request).df.tz_convert('America/New_York', level=1)

# data preprocessing for testing

# pct_change is profit from last close
test_df["pct_change"] = test_df["close"].pct_change()
# signal for when we want to be in or out of a stock
#bars_df["signal"] = np.where(bars_df["pct_change"] > 0, 1.0, 0.0)
# reaction is the signal diff
#bars_df["reaction"] = bars_df["signal"].diff()
# action is if we could perfectly predict the next close
#bars_df["action"] = bars_df["reaction"].shift(-1)
# these values are the high, low, and open as a percentage of the current close
test_df["high %"] = (test_df["high"] - test_df["close"])/test_df["close"]
test_df["low %"] = (test_df["low"] - test_df["close"])/test_df["close"]
test_df["open %"] = (test_df["open"] - test_df["close"])/test_df["close"]

# set timestamp as index, drop nan

test_df = test_df.droplevel(level=0).dropna()

test_df.info()
display(test_df.head())

In [ ]:
# Normalize test data set

test_df_scaled = scaler.fit_transform(test_df)

print(test_df_scaled)
print(len(test_df_scaled))

In [ ]:
# Test generator against real data

#lists to store generated and timestamps

column_names=[
    "Predicted Open",
    "Predicted High",
    "Predicted Low",
    "Predicted Close",
    "Predicted Volume",
    "Predicted Trade Count",
    "Predicted VWAP",
    "Predicted pct_change",
    "Predicted high %",
    "Predicted low %",
    "Predicted open %"
    ]

# sliding window

window_size = 1

# init lists to store generated data with associated timestamps

timestamps = []
predictions = []

In [ ]:
# shape of scaled test data

print(test_df_scaled.shape)

In [ ]:
predictions_df = pd.DataFrame(columns = column_names)

predictions_df[:] = 0

In [ ]:
print(test_df.index)
print(window_size)

In [ ]:
# Prediction loop

for i in range(0,len(test_df_scaled),window_size):
    print(i)
    # extract sliding window of real data

    input_data = test_df_scaled[i:i+window_size]

    # predict the next row using generator

    shaped_data = input_data.reshape(1,window_size,11)
    print(shaped_data)

    predicted_row = build_generator.predict(shaped_data)
    
    #predicted_row = build_generator.predict(input_data)

    #append generated row to list
    # predictions.append(predicted_row)

    # extract timestamp for the prediction

    if (i + window_size) < len(test_df):
        timestamp = test_df.index[i+window_size]
    else:
        timestamp = test_df.index[-1] + timedelta(days=1)
    
    print(timestamp)
    print(predicted_row.squeeze())
    predictions_df.loc[timestamp] = predicted_row.squeeze()
    print(predictions_df.head())
    

    # predictions_df = pd.concat([predictions_df,predicted_row],ignore_index=True)


In [ ]:
index = predictions_df.index

predictions_df_inverse_scaled = scaler.inverse_transform(predictions_df)
generated_data_df = pd.DataFrame(predictions_df_inverse_scaled, columns=column_names,index=index)
generated_data_df.head()



In [ ]:
test_df.head()

In [ ]:
bars_df = pd.concat([test_df,generated_data_df],axis=1)

In [ ]:
bars_df.head()

In [ ]:
test_shift_df = test_df.copy()

test_shift_df = test_shift_df.rename(columns={
    "open": "next open",
    "high": "next high",
    "low": "next low",
    "close": "next close",
    "volume": "next volume",
    "trade_count": "next trade_count",
    "vwap": "next vwap",
    "pct_change": "next pct_change",
    "high %": "next high %",
    "low %": "next low %",
    "open %": "next open %"
})

test_shift_df.head()


In [ ]:
test_shift_df = test_shift_df.shift(-1)

In [ ]:
bars_df = bars_df.dropna()
bars_df.head()

In [ ]:
bars_df = pd.concat([bars_df,test_shift_df],axis=1)
bars_df = bars_df.dropna()
bars_df.head()

In [ ]:
bars_df["High/Low Success"] = np.where(
    (bars_df["next close"] >= bars_df["Predicted Low"]),
    np.where(
        bars_df["close"] <= bars_df["Predicted High"], 1, 0
    ), 0
)

In [ ]:
bars_df["Predicted Close Delta"] = bars_df["Predicted Close"] - bars_df["close"]
bars_df["Next Close Delta"] = bars_df["next close"] - bars_df["close"]
bars_df["Close Product"] = bars_df["Next Close Delta"] * bars_df["Predicted Close Delta"]
bars_df["Close Success"] = np.where(
    bars_df["Close Product"] >= 0, 1, 0
)

In [ ]:
bars_df.head()

In [ ]:
bars_df = bars_df.rename_axis("Date")
bars_df.reset_index(inplace=True)
bars_df.head()

In [ ]:
actual_close = bars_df.hvplot.line(
    x="Date",
    y="next close",
)

In [ ]:
predicted_high = bars_df.hvplot.line(
    x="Date",
    y="Predicted high",
)

In [ ]:
predicted_low = bars_df.hvplot.line(
    x="Date",
    y="Predicted low",
)

In [ ]:
predicted_close = bars_df.hvplot.line(
    x="Date",
    y="Predicted close",
)

In [ ]:
actual_close * predicted_high * predicted_low * predicted_close

In [ ]:
bars_df.describe()

In [ ]:
bars_df["Action"] = np.where(
    bars_df["next close"] > bars_df["close"], 1, -1
)

In [ ]:
display(bars_df.head())
display(bars_df.tail())

In [ ]:
current_position = 0
starting_cash = 1000
current_cash = starting_cash
max_position = 50
for index, row in bars_df.iterrows():
    # Get the current action
    close = row["close"]
    action = row["Action"]

    # Take the action if possible
    if action > 0:
        if action + current_position <= max_position and action * close < current_cash:
            current_position += action
            current_cash -= action*close
        else:
            action = 0
    elif action < 0:
        if action + current_position >= 0:
            current_position += action
            current_cash += -action*close
        else:
            action = 0

    # Update enabled and position
    bars_df.loc[index, "Position"] = current_position
    bars_df.loc[index, "Cash"] = current_cash

In [ ]:
# Compute Holdings
bars_df["Holdings"] = bars_df["close"] * bars_df["Position"]

# Compute Profit
bars_df["Strategy Value"] = bars_df["Holdings"] + bars_df["Cash"]
starting_close = bars_df.iloc[0]["close"] 
display(f"starting_close {starting_close}")
starting_shares = math.floor(starting_cash / starting_close)
display(f"starting_shares {starting_shares}")
bars_df["Stock Value"] =  bars_df["close"] * starting_shares

# Compute Returns
bars_df["Stock Returns"] = bars_df["close"].pct_change()
bars_df["Strategy Returns"] = bars_df["Strategy Value"].pct_change()

# Compute Cumulative Daily Returns
bars_df["Stock Cumulative Returns"] = (bars_df["Stock Returns"] + 1).cumprod()
bars_df["Strategy Cumulative Returns"] = (bars_df["Strategy Returns"] + 1).cumprod()

In [ ]:
bars_df.tail()

In [ ]:
stock_cumulative_returns = bars_df.hvplot.line(
    x="Date",
    y="Stock Cumulative Returns",
)

In [ ]:
strategy_cumulative_returns = bars_df.hvplot.line(
    x="Date",
    y="Strategy Cumulative Returns",
)

In [ ]:
stock_cumulative_returns * strategy_cumulative_returns

In [ ]:
cash = bars_df.hvplot.line(
    x="Date",
    y="Cash",
)

In [ ]:
holdings = bars_df.hvplot.line(
    x="Date",
    y="Holdings",
)

In [ ]:
cash*holdings

In [ ]:
position = bars_df.hvplot.line(
    x="Date",
    y="Position",
)
position